[View in Colaboratory](https://colab.research.google.com/github/theamrzaki/seq2seq_Amazon_Reviews_tf_1.11/blob/master/Seq2Seq2.ipynb)

# Summarizing Text with Amazon Reviews

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 2.2.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


### Google Connect

## Insepcting the Data

In [ ]:
reviews = pd.read_csv("/content/drive/My Drive/telugu - telugu (4).csv")

In [ ]:
reviews.head(20)

,text,summary
0,ప్రమాదకర కరోనా వైరస్‌ భారత ఆర్మీకి సైతం పాకింద...,ఆర్మీ జవాన్‌కు కరోనా పాజిటివ్‌
1,సంక్షోభంలో పడిన ప్రైవేటు బ్యాంకు యస్‌ బ్యాంకు...,"యస్‌’ పునర్నిర్మాణ పథకం, త్వరలోనే ఆంక్షలు ఎత్త..."
2,ఏపీలో స్థానిక సంస్థల ఎన్నికలు వాయిదా వేయడాన్ని...,చంద్రబాబు క్షమాపణ చెప్పాలి
3,పాకిస్తాన్‌ మాజీ క్రికెటర్‌ షోయబ్‌ అక్తర్‌ చైన...,‘ఏది పడితే అది తిని ఈ మహమ్మారిని తెచ్చారు’
4,రాష్ట వ్యాప్తంగా సంచలన సృష్టించిన మిర్యాలగూడ ప...,తల్లి గిరిజను కలిసిన అమృతా ప్రణయ్‌
5,టీడీపీ అధ్యక్షుడు చంద్రబాబునాయుడు 13 జిల్లాలలో...,ఆ తర్వాత టీడీపీని ఓఎల్‌ఎక్స్‌లో పెట్టుకోవాలి
6,కరోనా వైరస్‌ ప్రపంచాన్ని వణికిస్తున్నప్పటికీ అ...,షెడ్యూల్‌ ప్రకారమే ఒలింపిక్స్‌: జపాన్‌ ప్రధాని
7,ఇండియన్‌ సూపర్‌ లీగ్‌ (ఐఎస్‌ఎల్‌) ఫుట్‌బాల్‌ ట...,కోల్‌కతాకే ఐఎస్‌ఎల్‌ కిరీటం
8,దేశంలో జరిగే అన్ని ఫుట్‌బాల్‌ మ్యాచ్‌లను ఈ నెల...,31 వరకు దేశంలో ‘నో’ ఫుట్‌బాల్‌
9,"భర్త, తల్లి ప్రవర్తన వల్ల మానసిక వేదనకు గురైన ...","తల్లి, భర్త మోసం.. నవవధువు ఆత్మహత్య"


In [ ]:
# Check for any nulls values
reviews.isnull().sum()

text       0
summary    0
dtype: int64

In [ ]:
reviews.head()

,text,summary
0,ప్రమాదకర కరోనా వైరస్‌ భారత ఆర్మీకి సైతం పాకింద...,ఆర్మీ జవాన్‌కు కరోనా పాజిటివ్‌
1,సంక్షోభంలో పడిన ప్రైవేటు బ్యాంకు యస్‌ బ్యాంకు...,"యస్‌’ పునర్నిర్మాణ పథకం, త్వరలోనే ఆంక్షలు ఎత్త..."
2,ఏపీలో స్థానిక సంస్థల ఎన్నికలు వాయిదా వేయడాన్ని...,చంద్రబాబు క్షమాపణ చెప్పాలి
3,పాకిస్తాన్‌ మాజీ క్రికెటర్‌ షోయబ్‌ అక్తర్‌ చైన...,‘ఏది పడితే అది తిని ఈ మహమ్మారిని తెచ్చారు’
4,రాష్ట వ్యాప్తంగా సంచలన సృష్టించిన మిర్యాలగూడ ప...,తల్లి గిరిజను కలిసిన అమృతా ప్రణయ్‌


In [ ]:
# Inspecting some of the reviews
for i in range(5):
    print("Review #",i+1)
    print(reviews.summary[i])
    print(reviews.text[i])
    print()

Review # 1
ఆర్మీ జవాన్‌కు కరోనా పాజిటివ్‌
ప్రమాదకర కరోనా వైరస్‌ భారత ఆర్మీకి సైతం పాకింది. పంజాబ్‌ సరిహద్దుల్లో విధులు నిర్వర్తిస్తున్న ఓ జవాను కరోనా లక్షణాలతో బాధపడుతుండగా.. అతన్ని సమీపంలోని ఆస్పత్రిలో చేర్పించారు. వైద్యుల పర్యవేక్షణలో పరీక్షలు నిర్వహించగా కరోనా పాజిటివ్‌ అని తేలింది. దీంతో అతన్ని మెరుగైన వైద్యం కోసం ఐసోలేషన్‌ సెంటర్‌కు తరలించారు. అయితే కరోనా సోకిన జవాను ఇటీవల ఇటలీ పర్యటను వెళ్లి వచ్చినట్టు అధికారులు బెబుతున్నారు. ఇటలీ పర్యటన అనంతరం మార్చి 11న మానేసర్‌లోని ఆర్మీ క్యాంపుకు వచ్చారని, ఈ నేపథ్యంలోనే వైరస్‌ లక్షణాలు కనిపించడంతో పరీక్షలు నిర్వహించామని వెల్లడించారు.

Review # 2
యస్‌’ పునర్నిర్మాణ పథకం, త్వరలోనే ఆంక్షలు ఎత్తివేత
సంక్షోభంలో పడిన  ప్రైవేటు బ్యాంకు యస్‌ బ్యాంకు పునర్నిర్మాణ పథకానికి కేబినెట్ ఆమోదం తెలిపింది. ఆర్‌బీఐ ప్రతిపాదనలకు మంత్రివర్గం ఆమోదం తెలిపిందని శుక్రవారం కేంద్ర కేబినెట్ సమావేశం అనంతరం ఏర్పాటు చేసిన మీడియా సమావేశంలో కేంద్ర ఆర్థికమంత్రి నిర్మలా సీతారామన్‌ వెల్లడించారు. తక్షణ మూలధన అవసరాల నిమిత్తం రూ. 1100 కోట్ల  నుంచి రూ. 6200 కోట్లకు పెంచినట్టు ఆమె ప

## Preparing the Data

In [ ]:
def clean_text(text):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    # Convert words to lower case
    #text = text.lower()
    # Replace contractions with their longer forms 
    text = text.split()
    textt=[]
    # Format words and remove unwanted characters
    for word in text:
      txt = re.sub(r'\u200c', ' ', word)
      textt.append(txt)
      text = " ".join(textt)
    # Optionally, remove stop words
    return text

In [4]:

# Clean the summaries and texts
clean_summaries = []
for summary in reviews.summary:
    clean_summaries.append(clean_text(summary))
print("Summaries are complete.")

clean_texts = []
for text in reviews.text:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

In [ ]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

Clean Review # 1
ఆర్మీ జవాన్ కు కరోనా పాజిటివ్ 
ప్రమాదకర కరోనా వైరస్  భారత ఆర్మీకి సైతం పాకింది. పంజాబ్  సరిహద్దుల్లో విధులు నిర్వర్తిస్తున్న ఓ జవాను కరోనా లక్షణాలతో బాధపడుతుండగా.. అతన్ని సమీపంలోని ఆస్పత్రిలో చేర్పించారు. వైద్యుల పర్యవేక్షణలో పరీక్షలు నిర్వహించగా కరోనా పాజిటివ్  అని తేలింది. దీంతో అతన్ని మెరుగైన వైద్యం కోసం ఐసోలేషన్  సెంటర్ కు తరలించారు. అయితే కరోనా సోకిన జవాను ఇటీవల ఇటలీ పర్యటను వెళ్లి వచ్చినట్టు అధికారులు బెబుతున్నారు. ఇటలీ పర్యటన అనంతరం మార్చి 11న మానేసర్ లోని ఆర్మీ క్యాంపుకు వచ్చారని, ఈ నేపథ్యంలోనే వైరస్  లక్షణాలు కనిపించడంతో పరీక్షలు నిర్వహించామని వెల్లడించారు.

Clean Review # 2
యస్ ’ పునర్నిర్మాణ పథకం, త్వరలోనే ఆంక్షలు ఎత్తివేత
సంక్షోభంలో పడిన ప్రైవేటు బ్యాంకు యస్  బ్యాంకు పునర్నిర్మాణ పథకానికి కేబినెట్ ఆమోదం తెలిపింది. ఆర్ బీఐ ప్రతిపాదనలకు మంత్రివర్గం ఆమోదం తెలిపిందని శుక్రవారం కేంద్ర కేబినెట్ సమావేశం అనంతరం ఏర్పాటు చేసిన మీడియా సమావేశంలో కేంద్ర ఆర్థికమంత్రి నిర్మలా సీతారామన్  వెల్లడించారు. తక్షణ మూలధన అవసరాల నిమిత్తం రూ. 1100 కోట్ల నుంచి రూ. 6200 కోట్లకు పెంచిన

In [ ]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [ ]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 27675


In [ ]:
word_counts


{'ఆర్మీ': 5,
 'జవాన్': 1,
 'కు': 273,
 'కరోనా': 319,
 'పాజిటివ్': 23,
 'యస్': 51,
 '’': 84,
 'పునర్నిర్మాణ': 7,
 'పథకం,': 2,
 'త్వరలోనే': 14,
 'ఆంక్షలు': 6,
 'ఎత్తివేత': 1,
 'చంద్రబాబు': 72,
 'క్షమాపణ': 2,
 'చెప్పాలి': 2,
 '‘ఏది': 1,
 'పడితే': 4,
 'అది': 44,
 'తిని': 6,
 'ఈ': 991,
 'మహమ్మారిని': 5,
 'తెచ్చారు’': 1,
 'తల్లి': 39,
 'గిరిజను': 4,
 'కలిసిన': 3,
 'అమృతా': 3,
 'ప్రణయ్': 9,
 'ఆ': 299,
 'తర్వాత': 114,
 'టీడీపీని': 4,
 'ఓఎల్': 4,
 'ఎక్స్': 27,
 'లో': 518,
 'పెట్టుకోవాలి': 1,
 'షెడ్యూల్': 22,
 'ప్రకారమే': 4,
 'ఒలింపిక్స్': 26,
 ':': 29,
 'జపాన్': 13,
 'ప్రధాని': 36,
 'కోల్': 12,
 'కతాకే': 1,
 'ఐఎస్': 3,
 'ఎల్': 26,
 'కిరీటం': 1,
 '31': 14,
 'వరకు': 103,
 'దేశంలో': 53,
 '‘నో’': 1,
 'ఫుట్': 12,
 'బాల్': 16,
 'తల్లి,': 3,
 'భర్త': 18,
 'మోసం..': 1,
 'నవవధువు': 1,
 'ఆత్మహత్య': 33,
 '24': 23,
 'గంటల్లో': 9,
 'వంద': 6,
 'మరణాలు..': 1,
 '2వేల': 4,
 'కేసులు': 34,
 'నా': 61,
 'ప్రతి': 39,
 'చరణం': 1,
 'ఒక': 142,
 'ఆత్మ': 2,
 'విమర్శ': 1,
 'ప్రేమ': 22,
 'కల': 8,
 'చెదిరింది..': 1,
 'డాక్ట

In [ ]:

embeddings_index = {}
with open('/content/drive/My Drive/cc.te.300.vec', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 1878289


In [ ]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 1

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 1830
Percent of words that are missing from vocabulary: 6.61%


I use a threshold of 20, so that words not in CN can be added to our word_embedding_matrix, but they need to be common enough in the reviews so that the model can understand their meaning.

In [ ]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 27675
Number of words we will use: 27679
Percent of words we will use: 100.01%


In [ ]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

27679


In [ ]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [ ]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 84319
Total number of UNKs in headlines: 0
Percent of words that are UNK: 0.0%


In [ ]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [ ]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
            counts
count  1007.000000
mean      4.467726
std       1.819024
min       1.000000
25%       3.000000
50%       4.000000
75%       5.000000
max      24.000000

Texts:
            counts
count  1007.000000
mean     80.265144
std      26.225286
min      30.000000
25%      61.000000
50%      76.000000
75%      94.000000
max     234.000000


In [ ]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

114.0
127.0
158.0


In [ ]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

6.0
7.0
9.0


In [ ]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [4]:
# takes a long time  , this is normal

# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 200
max_summary_length = 10
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match


## Building the Model

In [ ]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [ ]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_decoder

In [ ]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ , _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_decoder

In [ ]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    #initial_state = tf.contrib.seq2seq.AttentionWrapperState(enc_state[0],
    #                                                                _zero_state_tensors(rnn_size, 
    #                                                                                    batch_size, 
    #                                                                                    tf.float32)) 
    initial_state = dec_cell.zero_state(batch_size=batch_size,dtype=tf.float32).clone(cell_state=enc_state[0])

    with tf.variable_scope("decode"):
        training_decoder = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
        
        training_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_decoder = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
        
        inference_logits,_ ,_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                  output_time_major=False,
                                  impute_finished=True,
                                  maximum_iterations=max_summary_length)

    return training_logits, inference_logits

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [ ]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [4]:
# Set the Hyperparameters
epochs = 60
batch_size = 30
rnn_size = 128
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [4]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

In [ ]:
!pip install tensorflow==1.4.0

     |████████████████████████████████| 41.2MB 31.5MB/s 
     |████████████████████████████████| 1.7MB 41.8MB/s 
     |████████████████████████████████| 890kB 44.5MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=f19e508edb4f71aa8afaa4cf94a8acad7976d792c571666a6ca7cf4b91e207a3
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


## Training the Model

In [ ]:
# Subset the data for training
start = 0
end = start + 994
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 30
The longest text length: 179


In [ ]:
sorted_summaries
sorted_summaries_short

[[3, 835, 836, 837, 826],
 [1565, 1566, 1567],
 [12, 2559, 2560],
 [314, 315],
 [692, 693],
 [1498, 1499, 2, 1500],
 [1697, 1698, 121, 1699, 86, 1700],
 [2612, 2613],
 [1352, 365, 1353, 422, 584, 1354],
 [1762, 1692, 1763, 222, 127, 121, 575],
 [2189, 2190, 2191],
 [385, 609, 1090],
 [1637, 1638, 1639, 1640],
 [413, 414, 415, 416],
 [661, 662, 663, 664],
 [1071, 1072, 1073],
 [1116, 1117, 1118],
 [1660, 1661, 1662, 1663, 1664],
 [2151, 1580, 2152, 2148],
 [620, 335, 2446],
 [620, 335, 2446],
 [2160, 211, 2161, 1304],
 [1956, 1957, 1958, 222, 75, 1959],
 [632, 633, 634, 635],
 [1042, 1043, 1044, 564, 1045, 1046],
 [2028, 2029, 2030, 2031],
 [2705, 1356, 1750, 2706, 2707, 323],
 [697, 698],
 [215, 819, 820, 821, 822, 823],
 [1049, 28, 1050, 1051, 1052],
 [1742, 1743, 444, 1744, 1745, 1746, 1747, 1748],
 [2002, 2003, 2004],
 [1948, 1238, 1949, 179, 1950, 1951, 1952, 463],
 [110, 111, 112, 113, 114],
 [2325, 2326, 2327, 2328],
 [2407, 613, 2408],
 [476, 477, 2, 478, 356, 479],
 [78, 671, 6

In [ ]:
update_check

NameError: ignored

In [ ]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 5 # Check training loss after every 20 batches
stop_early = 0 
stop = 50  # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)+1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

  
tf.reset_default_graph
checkpoint = "/content/drive/My Drive/Colab Notebooks/best_model.ckpt" #300k sentence
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    # loader = tf.train.import_meta_graph(checkpoint + '.meta')
    # loader.restore(sess, checkpoint)
    #sess.run(tf.local_variables_initializer())

    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0
                
                saver = tf.train.Saver() 
                saver.save(sess, checkpoint)
                
            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
              
                  
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/30 Batch    5/33 - Loss:  9.280, Seconds: 2.85
Epoch   1/30 Batch   10/33 - Loss:  6.033, Seconds: 2.96
Average loss for this update: 7.465
New Record!
Epoch   1/30 Batch   15/33 - Loss:  6.126, Seconds: 3.56
Epoch   1/30 Batch   20/33 - Loss:  6.255, Seconds: 3.65
Average loss for this update: 5.949
New Record!
Epoch   1/30 Batch   25/33 - Loss:  5.689, Seconds: 4.39
Epoch   1/30 Batch   30/33 - Loss:  5.884, Seconds: 5.10
Epoch   2/30 Batch    5/33 - Loss:  5.617, Seconds: 3.19
Epoch   2/30 Batch   10/33 - Loss:  4.587, Seconds: 3.04
Average loss for this update: 5.114
New Record!
Epoch   2/30 Batch   15/33 - Loss:  4.836, Seconds: 3.58
Epoch   2/30 Batch   20/33 - Loss:  4.751, Seconds: 3.81
Average loss for this update: 4.592
New Record!
Epoch   2/30 Batch   25/33 - Loss:  4.417, Seconds: 4.40
Epoch   2/30 Batch   30/33 - Loss:  4.570, Seconds: 5.07
Epoch   3/30 Batch    5/33 - Loss:  5.276, Seconds: 3.08
Epoch   3/30 Batch   10/33 - Loss:  4.093, Seconds: 3.11
Average lo

In [ ]:
checkpoint = "/content/drive/My Drive/Colab Notebooks/best_model.ckpt" 

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    names = []
    [names.append(n.name) for n in loaded_graph.as_graph_def().node]
names

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/best_model.ckpt


['input',
 'targets',
 'learning_rate',
 'keep_prob',
 'summary_length',
 'Const',
 'max_dec_len',
 'text_length',
 'ReverseV2/axis',
 'ReverseV2',
 'embedding_lookup/params_0',
 'embedding_lookup/axis',
 'embedding_lookup',
 'embedding_lookup/Identity',
 'encoder_0/DropoutWrapperInit/Const',
 'encoder_0/DropoutWrapperInit/Const_1',
 'encoder_0/DropoutWrapperInit_1/Const',
 'encoder_0/DropoutWrapperInit_1/Const_1',
 'encoder_0/bidirectional_rnn/fw/fw/Rank',
 'encoder_0/bidirectional_rnn/fw/fw/range/start',
 'encoder_0/bidirectional_rnn/fw/fw/range/delta',
 'encoder_0/bidirectional_rnn/fw/fw/range',
 'encoder_0/bidirectional_rnn/fw/fw/concat/values_0',
 'encoder_0/bidirectional_rnn/fw/fw/concat/axis',
 'encoder_0/bidirectional_rnn/fw/fw/concat',
 'encoder_0/bidirectional_rnn/fw/fw/transpose',
 'encoder_0/bidirectional_rnn/fw/fw/sequence_length',
 'encoder_0/bidirectional_rnn/fw/fw/Shape',
 'encoder_0/bidirectional_rnn/fw/fw/strided_slice/stack',
 'encoder_0/bidirectional_rnn/fw/fw/strid

## Making Our Own Summaries

To see the quality of the summaries that this model can generate, you can either create your own review, or use a review from the dataset. You can set the length of the summary to a fixed value, or use a random value like I have here.

In [ ]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [ ]:
# Create your own review or use one from the dataset
#input_sentence = "I have never eaten an apple before, but this red one was nice. \
                  #I think that I will try a green apple next time."
#text = "తెలంగాణలో బీజేపీని అధికారంలోకి తీసుకురావడమే లక్ష్యంగా పనిచేస్తా. టీఆర్‌ఎస్, కేసీఆర్‌ ప్రజావ్యతిరేక విధానాలపై పోరాటం చేస్తా. శివాజీ వారసులుగా మేం వస్తాం. ఔరంగజేబు వారసులుగా ఎంఐఎం, టీఆర్‌ఎస్‌ వస్తోంది. విజయమో వీరస్వర్గమో తేల్చుకుంటా’ అని రాష్ట్ర బీజేపీ నూతన అధ్యక్షుడు బండి సంజయ్‌ స్పష్టం చేశారు. బీజేపీ రాష్ట్ర అధ్యక్షుడిగా   నియమితులయ్యాక తొలిసారి హైదరాబాద్‌ వచ్చిన ఆయనకు బీజేపీ శ్రేణులు పెద్ద ఎత్తున స్వాగతం పలికాయి. తొలుత గన్‌పార్క్‌ వద్ద తెలంగాణ అమరవీరుల స్తూపానికి ఆయన నివాళులు అర్పించి.. పార్టీ కార్యాలయానికి చేరుకున్నారు. అభినందన సభాస్థలిలో ఏర్పాటు చేసిన సర్దార్‌ వల్లభ్‌భాయ్‌ పటేల్, సుష్మాస్వరాజ్‌ చిత్రపటాలకు నివాళులు అర్పించారు. కాగా, సంజయ్‌ బీజేపీ రాష్ట్ర అధ్యక్షుడిగా ఆదివారం బాధ్యతలు స్వీకరించలేదు. ముహూర్తం చూసుకుని స్వీకరించనున్నట్లు సమాచారం."
#random = np.random.randint(0,len(clean_texts))
checkpoint = "/content/drive/My Drive/Colab Notebooks/best_model.ckpt"
for j in range(0,len(clean_texts)):

  input_sentence = clean_texts[j]
  text = text_to_seq(input_sentence)
#text = text_to_seq(txt)

  loaded_graph = tf.Graph()
  with tf.Session(graph=loaded_graph) as sess:

    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(2,10)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
  pad = vocab_to_int["<PAD>"] 

  print('Original Text:', input_sentence)
  print('Original summary:', clean_summaries[j])#clean_summaries[random]

  print('\nText')
  print('  Word Ids:    {}'.format([i for i in text]))
  print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

  print('\nSummary')
  print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
  print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

Streaming output truncated to the last 5000 lines.
  Input Words: నిరాధార వార్తలను అరికట్టడమే లక్ష్యంగా, ఎన్నికలకు సంబంధించి తప్పుడు సమాచారం పోస్ట్ చేయడాన్ని యూట్యూబ్ లో నిషేధించనున్నట్లు గూగుల్ సంస్థ ప్రతినిధులు తెలిపారు. ప్రభుత్వ ఉద్యోగులు మరణించారనిగానీ, ఎన్నికల తేదీ వ్యవహారంలో తప్పుడు సమాచారంగానీ యూట్యూబ్ లో పోస్ట్ చేస్తే దాన్ని తొలగిస్తామని ఆ సంస్థ చెప్పింది. తాము నియమించిన ట్రస్ట్ అండ్ సేఫ్టీ టీమ్ లు ఆయా వీడియోలను నిరంతరం పరిశీలిస్తుంటాయని చెప్పింది. వార్తలకు నమ్మదగిన స్థానం కలిగిన దానిగా యూట్యూబ్ ను మార్చనున్నట్లు తెలిపింది.

Summary
  Word Ids:       [1862, 1875, 1876, 1877]
  Response Words: తప్పుడు సమాచారాన్ని నిషేధించనున్న యూట్యూబ్
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/best_model.ckpt
Original Text: టిక్ టాక్  యాప్ ను భారతీయులు అధికంగా వినియోగిస్తున్నారు. రోజురోజుకు టిక్ టాక్  యాప్ ను ఉపయోగిస్తున్నవారి సంఖ్య పెరుగుతోంది. యూజర్లు తమ వీడియోల ద్వారా ప్రతిభను బయటపెడుతున్నారు. అదేవిధంగా చాలా మంది ఈ టిక్ టాక్  వీడియోల్లో డాన్స్ లు, పాటలు

In [1]:
from nltk.translate.bleu_score import sentence_bleu

In [1]:
so = []
for i in news['text']:
  out = summarize(i)
  so.append(out)

In [1]:
docs = news['text']
sums = news['summary']

In [2]:
bl = []
for i,j in zip(sums,so):
  reference = [i.split()]
  candidate = j.split()
  score = sentence_bleu(reference, candidate,smoothing_function=None)
  bl.append(score)

In [2]:
av_bl = sum(bl) / len(bl)

In [4]:
av_bl

0.76538

In [ ]:
# Create your own review or use one from the dataset
#input_sentence = "I have never eaten an apple before, but this red one was nice. \
                  #I think that I will try a green apple next time."
  txt = "కరోనా వైరస్‌ను సృష్టించి వ్యాప్తి చేసిందని ఆరోపిస్తూ చైనాపై 20 లక్షల కోట్ల డాలర్ల పరిహారం కోరుతూ వాషింగ్టన్‌కు చెందిన న్యాయవాది లారీ క్లేమన్‌ కేసు దాఖలు చేశారు. క్లేమన్‌కు చెందిన ఫ్రీడం వాచ్‌ అండ్‌ బజ్‌ ఫోటోస్‌ అనే సంస్థ టెక్సాస్‌లోని అమెరికా జిల్లా కోర్టులో ఈ కేసు నమోదు చేసింది. జీవరసాయన ఆయుధంగా కరోనా వైరస్‌ను చైనా డిజైన్‌ చేసిందని క్లేమన్‌ ఆరోపించారు. ఈ వైరస్‌ను సృష్టించిన చైనా అమెరికా చట్టంతో పాటు అంతర్జాతీయ చట్టాలు, ఒప్పందాలు, నిబంధనలను ఉల్లంఘించిందని అన్నారు."
#random = np.random.randint(0,len(clean_texts))
  checkpoint = "/content/drive/My Drive/Colab Notebooks/best_model.ckpt"
#for j in range(0,len(clean_texts)):

  #input_sentence = clean_texts[j]
  #text = text_to_seq(input_sentence)
  text = text_to_seq(txt)

  loaded_graph = tf.Graph()
  with tf.Session(graph=loaded_graph) as sess:

    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(2,10)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
  pad = vocab_to_int["<PAD>"] 

  print('Original Text:', txt)
  #print('Original summary:', clean_summaries[j])#clean_summaries[random]

  print('\nText')
  print('  Word Ids:    {}'.format([i for i in text]))
  print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

  print('\nSummary')
  print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
  print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/best_model.ckpt
Original Text: కరోనా వైరస్‌ను సృష్టించి వ్యాప్తి చేసిందని ఆరోపిస్తూ చైనాపై 20 లక్షల కోట్ల డాలర్ల పరిహారం కోరుతూ వాషింగ్టన్‌కు చెందిన న్యాయవాది లారీ క్లేమన్‌ కేసు దాఖలు చేశారు. క్లేమన్‌కు చెందిన ఫ్రీడం వాచ్‌ అండ్‌ బజ్‌ ఫోటోస్‌ అనే సంస్థ టెక్సాస్‌లోని అమెరికా జిల్లా కోర్టులో ఈ కేసు నమోదు చేసింది. జీవరసాయన ఆయుధంగా కరోనా వైరస్‌ను చైనా డిజైన్‌ చేసిందని క్లేమన్‌ ఆరోపించారు. ఈ వైరస్‌ను సృష్టించిన చైనా అమెరికా చట్టంతో పాటు అంతర్జాతీయ చట్టాలు, ఒప్పందాలు, నిబంధనలను ఉల్లంఘించిందని అన్నారు.

Text
  Word Ids:    [3, 875, 149, 1870, 3450, 8966, 14183, 27675, 1774, 881, 1945, 5008, 19749, 9276, 16816, 2, 1663, 9115, 2203, 27675, 567, 9089, 3155, 27675, 2, 1663, 27675, 27675, 3599, 27675, 27675, 3417, 4416, 17586, 3077, 717, 4864, 17056, 19, 567, 568, 4000, 27675, 27675, 3, 875, 149, 732, 3993, 8966, 27675, 3239, 19, 875, 149, 3186, 732, 717, 27675, 152, 2970, 27675, 27675, 5056, 27675, 3170]
  Input Word

I hope that you found this project to be rather interesting and informative. One of my main recommendations for working with this dataset and model is either use a GPU, a subset of the dataset, or plenty of time to train your model. As you might be able to expect, the model will not be able to make good predictions just by seeing many news Articles, it needs so see the news Articles many times to be able to understand the relationship between words and between descriptions & summaries. 

In short, I'm pleased with how well this model performs. After creating numerous summaties and checking those from the dataset, I can happily say that most of the generated summaries are appropriate, some of them are great, and some of them make mistakes. I'll try to improve this model and if it gets better, I'll update my GitHub.

Thanks for reading!